In [1]:
from keras import layers
from keras import models

Using TensorFlow backend.


# 1. 소개

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                36928     
__________

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [6]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data('mnist.npz')

train_images = train_images.reshape((-1, 28, 28, 1))
train_images = train_images.astype('float32') / 255.

test_images = test_images.reshape((-1, 28, 28, 1))
test_images = test_images.astype('float32') / 255.

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 10s 162us/step - loss: 0.1874 - acc: 0.9423
Epoch 2/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.0500 - acc: 0.9845
Epoch 3/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.0349 - acc: 0.9890
Epoch 4/5
60000/60000 [==============================] - 4s 61us/step - loss: 0.0268 - acc: 0.9917
Epoch 5/5
60000/60000 [==============================] - 4s 64us/step - loss: 0.0208 - acc: 0.9935


In [8]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

10000/10000 [==============================] - 0s 41us/step


0.99299999999999999

## 1. 합성곱 연산

- Dense 층은 입력 특성 공간에 있는 전역 패턴을 학습하지만 합성곱 층은 지역 패턴을 학습한다. 이 핵심 특징은 컨브넷에 2가지 흥미로운 성질을 제공한다.

    1. 학슴된 패턴은 평행 이동 불변성을 가집니다. 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳에서도 이 패턴을 인식할 수 있다. 근본적으로 우리가 보는 세상은 평행 이동으로 인해 다르게 인식되지 않습니다.
    
    2. 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있습니다. 첫 번째 합성곱 층이 에지 같은 지역 패턴을 학습한다. 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습한다. 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있습니다. 근본적으로 우리가 보는 세상은 공간적 계층 구조를 가집니다.


- 필터 하나의 크기는 (patch_height, patch_width, input_depth)입니다. 첫 번째 합성곱은 (3, 3, 1) 크기의 필터를 32개 적용하고 두 번째 합성곱은 (3, 3, 32) 크기 필터를 64개 적용합니다.

- Conv2D에서 padding은 매개변수로 설정할 수 있따. valid는 패딩을 사용하지 않는다는 뜻입니다. same은 입력과 동일한 높이와 넓이를 가진 출력을 만들기 위해 패딩한다. padding 매개변수 기본값은 valid이다.

## 2. 맥스 풀링 연산

In [9]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model_no_max_pool.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


### Maxpool을 사용하지 않을 때 문제

1) 특성의 공간적 계층 구조를 학습하는 데 도움이 되지 않는다. 세 번째 층의 3x3 윈도우는 초기 입력의 7x7 윈도우 영역 정보만 담고 있습니다. 컨브넷에 의해 학습된 고수준 패턴은 초기 입력에 관한 정보가 아주 적어 숫자 분류를 학습하기에 충분하지 않을 것입니다. 마지막 합성곱 층의 특징이 전체 입력에 대한 정보를 가지고 있어야 한다.

2) 최종 특성 맵은 22x22x64 = 30,976개의 가중치를 가집니다. Dense와 연결하면 더 커진다. 작은 모델 치고는 너무 많은 가중치이고 심각한 과대 적합이 발생할 것이다.

- 간단히 말해서 다운샘플링을 하는 이유는 처리할 특성 맵의 가중치 개수를 줄이고, 연속적인 합성 곱층이 점점 커진 윈도우를 통해 바라보도록 만들어 필터의 공간적인 계층 구조를 구성합니다.